# 실습 08: 표 검출 및 데이터 추출

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/leecks1119/document_ai_lecture/blob/master/notebooks/Lab08_표검출.ipynb)

## 🎯 학습 목표
- 이미지에서 표 영역 검출
- 표 구조 분석 (행/열)
- 표 데이터를 DataFrame으로 변환

## ⏱️ 소요 시간: 40분
## 📊 난이도: ⭐⭐⭐⭐☆


In [ ]:
# 한글 폰트 설치 및 설정 (Colab용)
!apt-get install -y fonts-nanum fonts-nanum-coding fonts-nanum-extra
!fc-cache -fv

# matplotlib 한글 폰트 설정
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
fontprop = fm.FontProperties(fname=font_path, size=10)
plt.rc('font', family=fontprop.get_name())
plt.rc('axes', unicode_minus=False)

print("✅ 한글 폰트 설정 완료!")


In [ ]:
# 환경 설정
!pip install -q opencv-python-headless paddlepaddle paddleocr
!apt-get install -y tesseract-ocr tesseract-ocr-kor

import cv2
import numpy as np
import pandas as pd
from paddleocr import PaddleOCR
import matplotlib.pyplot as plt


In [ ]:
# 표가 포함된 샘플 이미지 생성
from PIL import Image, ImageDraw, ImageFont

# 한글 폰트 로드
try:
    font_title = ImageFont.truetype("C:\\Windows\\Fonts\\malgunbd.ttf", 28)  # 굵은 글씨
    font_cell = ImageFont.truetype("C:\\Windows\\Fonts\\malgun.ttf", 18)
except:
    try:
        font_title = ImageFont.truetype("/usr/share/fonts/truetype/nanum/NanumGothicBold.ttf", 28)
        font_cell = ImageFont.truetype("/usr/share/fonts/truetype/nanum/NanumGothic.ttf", 18)
    except:
        font_title = ImageFont.load_default()
        font_cell = ImageFont.load_default()

img = Image.new('RGB', (800, 400), color='white')
draw = ImageDraw.Draw(img)

# 제목
draw.text((30, 20), "견적서", fill='black', font=font_title)

# 표 그리기
table_data = [
    ["품목", "수량", "단가", "금액"],
    ["노트북", "10", "1,500,000", "15,000,000"],
    ["모니터", "20", "300,000", "6,000,000"],
    ["키보드", "30", "50,000", "1,500,000"],
]

x_start, y_start = 50, 70
cell_width = 180
cell_height = 40

# 셀 그리기
for i, row in enumerate(table_data):
    for j, cell in enumerate(row):
        x = x_start + j * cell_width
        y = y_start + i * cell_height
        # 테두리
        draw.rectangle([x, y, x+cell_width, y+cell_height], outline='black', width=2)
        # 텍스트
        draw.text((x+10, y+10), cell, fill='black', font=font_cell)

img.save('table_doc.jpg')
print("✅ 표 이미지 생성 완료!")
img


## 💡 표 영역 검출의 중요성

### 문제: 제목과 표 데이터가 섞임
표 외부의 텍스트(제목, 날짜 등)가 표 데이터로 포함되어 잘못된 결과 생성

### 해결: 표 영역만 정확히 검출
```python
# 1. 표 구조 검출 (수평선 + 수직선)
table_structure = cv2.add(horizontal_lines, vertical_lines)

# 2. Contours로 표 경계 찾기
contours = cv2.findContours(table_structure, ...)
table_bbox = cv2.boundingRect(largest_contour)

# 3. 표 영역 내부의 텍스트만 필터링
if (table_x <= text_x <= table_x + table_w and 
    table_y <= text_y <= table_y + table_h):
    # 표 안의 텍스트만 추가
```

### 결과
✅ 표 외부 텍스트 제외 → 정확한 표 데이터 추출


In [ ]:
# OpenCV로 표 영역 검출
img = cv2.imread('table_doc.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 이진화
_, binary = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV)

# 수평선 검출
horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (40, 1))
horizontal_lines = cv2.morphologyEx(binary, cv2.MORPH_OPEN, horizontal_kernel, iterations=2)

# 수직선 검출
vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 40))
vertical_lines = cv2.morphologyEx(binary, cv2.MORPH_OPEN, vertical_kernel, iterations=2)

# 표 구조 결합
table_structure = cv2.add(horizontal_lines, vertical_lines)

# 🔍 표 경계 검출 (Contours)
contours, _ = cv2.findContours(table_structure, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 가장 큰 contour가 표 영역
table_contour = max(contours, key=cv2.contourArea)
x, y, w, h = cv2.boundingRect(table_contour)

# 표 영역 저장 (나중에 OCR 시 사용)
table_bbox = (x, y, w, h)

# 시각화
img_with_bbox = img.copy()
cv2.rectangle(img_with_bbox, (x, y), (x+w, y+h), (0, 255, 0), 3)

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

axes[0, 0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
axes[0, 0].set_title('원본 이미지', fontproperties=fontprop, fontsize=14)
axes[0, 0].axis('off')

axes[0, 1].imshow(cv2.cvtColor(img_with_bbox, cv2.COLOR_BGR2RGB))
axes[0, 1].set_title('표 영역 검출 (초록색 박스)', fontproperties=fontprop, fontsize=14)
axes[0, 1].axis('off')

axes[1, 0].imshow(horizontal_lines, cmap='gray')
axes[1, 0].set_title('수평선 검출', fontproperties=fontprop, fontsize=14)
axes[1, 0].axis('off')

axes[1, 1].imshow(vertical_lines, cmap='gray')
axes[1, 1].set_title('수직선 검출', fontproperties=fontprop, fontsize=14)
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

print(f"✅ 표 구조 검출 완료!")
print(f"📍 표 영역 좌표: x={x}, y={y}, width={w}, height={h}")


In [ ]:
# PaddleOCR로 텍스트 추출 (표 영역만)
ocr = PaddleOCR(use_textline_orientation=True, lang='korean')
result = ocr.ocr('table_doc.jpg', cls=True)

# 표 영역 좌표 (이전 셀에서 검출한 table_bbox 사용)
table_x, table_y, table_w, table_h = table_bbox

# ⚠️ 핵심: 표 영역 내부의 텍스트만 필터링
ocr_results = []
for line in result[0]:
    bbox = line[0]
    text = line[1][0]
    score = line[1][1]
    
    # 중심점 계산
    x_center = (bbox[0][0] + bbox[2][0]) / 2
    y_center = (bbox[0][1] + bbox[2][1]) / 2
    
    # 🔍 표 영역 내부에 있는 텍스트만 추가
    if (table_x <= x_center <= table_x + table_w and 
        table_y <= y_center <= table_y + table_h):
        ocr_results.append({
            'text': text,
            'x': x_center,
            'y': y_center,
            'confidence': score
        })

# Y 좌표로 먼저 정렬 (행), 그 다음 X 좌표로 정렬 (열)
ocr_results_sorted = sorted(ocr_results, key=lambda r: (r['y'], r['x']))

print("\n📋 추출된 텍스트 (표 영역만, 순서대로):")
print("="*70)
for i, item in enumerate(ocr_results_sorted, 1):
    print(f"{i:2d}. {item['text']:20s} (신뢰도: {item['confidence']:.2%}, x={item['x']:.0f}, y={item['y']:.0f})")
print("="*70)
print(f"\n✅ 총 {len(ocr_results_sorted)}개의 셀 검출 (표 영역만)")


In [ ]:
# DataFrame으로 변환 (자동 행/열 구조 분석)
texts = [item['text'] for item in ocr_results_sorted]

# 🔍 Y 좌표 기반으로 행 그룹화 (같은 행은 Y 좌표가 비슷함)
rows_grouped = []
current_row = []
current_y = None
y_threshold = 20  # Y 좌표 차이 임계값

for item in ocr_results_sorted:
    if current_y is None:
        current_y = item['y']
        current_row = [item['text']]
    elif abs(item['y'] - current_y) < y_threshold:
        # 같은 행
        current_row.append(item['text'])
    else:
        # 새로운 행
        rows_grouped.append(current_row)
        current_row = [item['text']]
        current_y = item['y']

# 마지막 행 추가
if current_row:
    rows_grouped.append(current_row)

print("\n🔍 행별 그룹화 결과:")
for i, row in enumerate(rows_grouped, 1):
    print(f"  행 {i}: {row}")

# DataFrame 생성
if len(rows_grouped) >= 2:
    df = pd.DataFrame(rows_grouped[1:], columns=rows_grouped[0])
    
    print("\n📊 표 데이터 (DataFrame):")
    print("="*80)
    print(df.to_string(index=False))
    print("="*80)
    
    # CSV로 저장
    df.to_csv('extracted_table.csv', index=False, encoding='utf-8-sig')
    print("\n✅ CSV 저장 완료: extracted_table.csv")
    print(f"   - 행 수: {len(df)}개")
    print(f"   - 열 수: {len(df.columns)}개")
else:
    print("\n⚠️ 표 데이터가 충분하지 않습니다.")
